# DetecTreeRGB Reprojecting

This loads the jsons files that are predictions from a trained model. It reads the files and reorganised them back into a geojson format and writes a file with this name.

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# install necessary geospatial packages

!pip -q install rasterio
!pip -q install fiona
!pip -q install geopandas
!pip -q install pycrs
!pip -q install descartes 
!pip -q install pypng

     |████████████████████████████████| 19.3 MB 1.2 MB/s 
     |████████████████████████████████| 16.7 MB 440 kB/s 
     |████████████████████████████████| 1.0 MB 5.4 MB/s 
     |████████████████████████████████| 6.3 MB 29.8 MB/s 
     |████████████████████████████████| 48 kB 2.2 MB/s 


In [ ]:
# necessary basic libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import png
import glob

# geospatial libraries
import rasterio
import geopandas
from geopandas.tools import sjoin
import fiona
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import descartes

# import more geospatial libraries
import rasterio
from rasterio.transform import from_origin
import rasterio.features
import pycocotools.mask as mask_util

import fiona

from shapely.geometry import shape, mapping, box
from shapely.geometry.multipolygon import MultiPolygon

# Reprojecting json back to geojson

In [ ]:
# Code to convert RLE data from the output instances into Polygons, a small about of info is lost but is fine.
# https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py <-- found here

def polygonFromMask(maskedArr):

    contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    segmentation = []
    for contour in contours:
        # Valid polygons have >= 6 coordinates (3 points)
        if contour.size >= 6:
            segmentation.append(contour.flatten().tolist())
    RLEs = mask_util.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
    RLE = mask_util.merge(RLEs)
    # RLE = mask.encode(np.asfortranarray(maskedArr))
    area = mask_util.area(RLE)
    [x, y, w, h] = cv2.boundingRect(maskedArr)

    return segmentation[0] #, [x, y, w, h], area

In [ ]:
### Editing format to geojson 
### Need to edit the image_link, the directory and then the EPSG value

buffer=20
resolution = 0.6  # this is the resolution of the tif originally used to tile the images
image_link= 'gdrive/MyDrive/JamesHirst/NY/LargeArea_images/naip_buf/naip_buf.tif'

directory= 'gdrive/MyDrive/JamesHirst/NY/LargeArea_images/naip_buf/naip_buf_predictions/'
entries = os.listdir(directory)

# need this data file to give the bounds so the image can be rescaled correctly if at the edge
data = rasterio.open(image_link)

for file in entries:
  if ".json" in file: 
    #create a geofile for each tile --> the EPSG value might need to be changed.
    geofile = {"type": "FeatureCollection", "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::26917"}}, "features":[]}

    # create a dictionary for each file to store data used multiple times
    img_dict = {}
    img_dict["filename"] = file

    file_mins = file.replace(".json", "")
    file_mins_split = file_mins.split("_")
    img_dict["minx"]= file_mins_split[1]
    img_dict["miny"]= file_mins_split[2]

    # load the json file we need to convert into a geojson
    with open(directory+img_dict["filename"]) as prediction_file:
      datajson = json.load(prediction_file)
    #print(datajson)
 
    img_dict["width"] = datajson[0]["segmentation"]["size"][0]
    img_dict["height"] = datajson[0]["segmentation"]["size"][1]
    print(img_dict)

    # json file is formated as a list of segmentation polygons so cycle through each one
    for crown_data in datajson:
      #just a check that the crown image is correct
      if img_dict["minx"]+'_'+img_dict["miny"] in crown_data["image_id"]:
        crown = crown_data["segmentation"]

        # changing the coords from RLE format so can be read as numbers, here the numbers are
        # integers so a bit of info on position is lost
        mask_of_coords = mask_util.decode(crown)
        crown_coords = polygonFromMask(mask_of_coords)
        rescaled_coords = []

        # coords from json are in a list of [x1, y1, x2, y2,... ] so convert them to [[x1, y1], ...]
        # format and at the same time rescale them so they are in the correct position for QGIS
        for c in range(0, len(crown_coords), 2): 
          x_coord=crown_coords[c]
          y_coord=crown_coords[c+1]

          # rescaling the coords depending on where the tile is in the original image, note the correction
          # factors have been manually added as outputs did not line up with predictions from training script
          if int(img_dict["minx"]) == data.bounds[0] and int(img_dict["miny"]) == data.bounds[1]:
            #print("Bottom Corner")
            x_coord = (x_coord)*resolution+int(img_dict["minx"])
            y_coord = (img_dict["height"]-y_coord)*resolution+int(img_dict["miny"])
          elif int(img_dict["minx"]) == data.bounds[0]: 
            #print("Left Edge")
            x_coord = (x_coord)*resolution+int(img_dict["minx"])
            y_coord = (img_dict["height"]-y_coord-buffer)*resolution+int(img_dict["miny"])+12
          elif int(img_dict["miny"]) == data.bounds[1]:
            #print("Bottom Edge")
            x_coord = (x_coord-2*buffer)*resolution+int(img_dict["minx"])+3.5
            y_coord = (img_dict["height"]-y_coord-2*buffer)*resolution+int(img_dict["miny"])+3.5
          else:
            #print("Anywhere else")
            x_coord = (x_coord-2*buffer)*resolution+int(img_dict["minx"])+3.5
            y_coord = (img_dict["height"]-y_coord-2*buffer)*resolution+int(img_dict["miny"])+3.5

          rescaled_coords.append([x_coord,y_coord])

        geofile["features"].append({"type": "Feature", "properties": {}, "geometry" :{"type": "Polygon", "coordinates": [rescaled_coords]}})

    # Check final form is correct - compare to a known geojson file if error appears.
    print(geofile)

    output_geo_file = directory + img_dict["filename"].replace('.json','.geojson')
    print(output_geo_file)
    with open(output_geo_file, "w") as dest:
      json.dump(geofile,dest)


{'filename': 'prediction_680082_4753854.json', 'minx': '680082', 'miny': '4753854', 'width': 401, 'height': 401}
{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::26917'}}, 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[680124.5, 4753976.9], [680123.9, 4753976.3], [680123.3, 4753976.3], [680122.1, 4753975.1], [680122.1, 4753974.5], [680121.5, 4753973.9], [680121.5, 4753969.7], [680122.1, 4753969.1], [680122.1, 4753967.9], [680123.3, 4753966.7], [680123.3, 4753966.1], [680124.5, 4753964.9], [680125.1, 4753964.9], [680125.7, 4753964.3], [680126.3, 4753964.3], [680126.9, 4753963.7], [680129.9, 4753963.7], [680130.5, 4753964.3], [680131.7, 4753964.3], [680132.3, 4753964.9], [680132.9, 4753964.9], [680134.1, 4753966.1], [680134.1, 4753966.7], [680134.7, 4753967.3], [680134.7, 4753973.3], [680134.1, 4753973.9], [680134.1, 4753974.5], [680133.5, 4753974.5], [680131.1, 4753976.9]]]}}, {'type